In [ ]:
!pip install mediapipe opencv-python pandas


import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import io
from PIL import Image




def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📸 Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(track => track.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg', quality);
    }
  ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def extract_keypoints(results):
    if results.multi_hand_landmarks:
        hand = results.multi_hand_landmarks[0]
        keypoints = []
        for lm in hand.landmark:
            keypoints.append(lm.x)
            keypoints.append(lm.y)
        return keypoints
    else:
        return None


def create_dataset_folder(label):
    folder = f"/content/dataset/{label}"
    os.makedirs(folder, exist_ok=True)
    return folder


# 🔁 Repeat this block for each image you want to capture
label = "namaste"  # Change this label each time you record a new sign
filename = take_photo()
image = cv2.imread(filename)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = hands.process(image_rgb)

if results.multi_hand_landmarks:
    mp_drawing.draw_landmarks(image, results.multi_hand_landmarks[0], mp_hands.HAND_CONNECTIONS)
    keypoints = extract_keypoints(results)
    if keypoints:
        folder = create_dataset_folder(label)
        filepath = os.path.join(folder, f"{label}_{np.random.randint(1000)}.csv")
        pd.DataFrame([keypoints]).to_csv(filepath, index=False)
        print(f"✅ Keypoints saved to {filepath}")
    else:
        print("⚠️ Hand detected but keypoints missing.")
else:
    print("🚫 No hand detected.")

# Show the captured and annotated image
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


import glob

def load_dataset(data_dir="/content/dataset"):
    all_data = []
    all_labels = []

    for label_folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, label_folder)
        for file in glob.glob(f"{folder_path}/*.csv"):
            df = pd.read_csv(file)
            if df.shape[1] == 42:  # Make sure it’s a full hand keypoints file
                all_data.append(df.values[0])
                all_labels.append(label_folder)

    return np.array(all_data), np.array(all_labels)

X, y = load_dataset()
print("Shape of X:", X.shape)
print("Labels:", np.unique(y))


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))


import cv2
import mediapipe as mp
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io
import numpy as np

# MediaPipe hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

def take_photo_for_prediction(filename='test.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();

            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


filename = take_photo_for_prediction()
image = cv2.imread(filename)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
results = hands.process(image_rgb)

if results.multi_hand_landmarks:
    hand_landmarks = results.multi_hand_landmarks[0]
    keypoints = []
    for lm in hand_landmarks.landmark:
        keypoints.extend([lm.x, lm.y])

    # Ensure it's 42 features
    if len(keypoints) == 42:
        prediction = clf.predict([keypoints])[0]
        print("✅ Predicted Sign:", prediction)
    else:
        print("⚠️ Incomplete hand detected.")
else:
    print("❌ No hand detected.")

# Move into your repo directory (if not already there)
%cd your-repo

# Stage changes (this adds your file to Git tracking)
!git add .

# Commit changes with a message
!git commit -m "Added notebook from Colab"

# Push to GitHub (you'll be prompted for GitHub username and token)
!git push origin main
